## Business problem

Port terminals constantly strive to improve the efficiency of their operations through a careful
management of their berth facilities, machinery and personnel. 



The most important variables when planning terminal operations is knowing which vessels will arrive at the terminal and when.


MarineTraffic aims at being the best visibility providers by providing up to date vessel tracking data (using AIS), as well as additional derived information such as the estimated time of arrival (ETA) of a vessel to a port of interest.


AIS messages contain information on the port that the vessel is traveling to as well as the estimated time of arrival.

However, since ports may consist of more than one terminal, the exact terminal that the
vessel will visit is not known in advance making it difficult for MarineTraffic to assign future arrivals to terminals which, in turn, limits the ability to measure terminal congestion and calculate more accurate terminal arrival times.



A model which predicts the terminal a vessel will travel to has the potential to help all parties involved in a port call to plan their operations more effectively

# Data Description

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of **Hamburg and Port of Los Angeles**. The dataset contains the following fields

A dataset has been extracted containing container calls at terminals that took place during the past 3
years for the Port of Hamburg and Port of Los Angeles. The dataset contains the following fields;


* **last_port**: Port where the *last* terminal call by vessel is recorded.



* **last_terminal**: The immediately previous *terminal* call of the vessel.


* **last_terminal_doc_timestamp**: Timestamp of previous terminal call.


* **current_port**: Port where the current terminal call by vessel is recorded.


* **current_terminal**: Current terminal call of the vessel.


* **shipname**: Name of the vessel.


* **dock_timestamp**: Timestamp of current terminal call.


* **GRT**: Vessel capacity (gross tonnage unit).


* **TEU**: Vessel capacity (twenty-foot equivalent Unit).


* **length**: Vessel length.


* **width**: Vessel width.

# Goals and Deliverable


The goal of this task is to implement & evaluate the accuracy of a solution that predicts the terminal
that a vessel will call.
**The product team claims that the history of terminals visited by a vessel in the past is a critical factor
that should be incorporated into the model**.

Some important steps that your solution would be expected to address and describe are the following:
* What features have you finally selected and engineered for your modeling approach? What led you to these choices? Why & how have you processed them?
* Which features seem to be the most important & how did you evaluate their importance?
* Do your findings agree with the product team’s insights discussed above? Before developing a ML model, how would you evaluate the importance/predictive power of one of the productidentified features as an independent variable?
* What type of prediction/training model have you chosen and why?
* How well does your predictive solution perform in terms of predicting the terminal a vessel will call?
* What different metrics/graphs can you use in order to understand when & why the algorithm fails/succeeds?
* What would be your baseline (i.e. a “naive” approach) to compare against?

# Solution Procedure

### problem understanding & EDA

In order to understand the problem we have to dive into the data

In [ ]:
import pandas as pd
import numpy as np
from pprint import pprint
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import PolynomialFeatures
import warnings
from tqdm import tqdm
from scikitplot.estimators import plot_learning_curve
warnings.filterwarnings('ignore')

In [ ]:
def heatmap(y_test, y_pred, label_no=11):
    cm = confusion_matrix(y_test, y_pred)

    # Normalize the confusion matrix
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    # Define the labels for the x and y axes
    labels = list(range(label_no))

    # Create the heatmap
    sns.heatmap(cm_norm, annot=cm, cmap='Blues', xticklabels=labels, yticklabels=labels)

    # Add labels and title
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Classification Heatmap')

In [ ]:
raw_data = pd.read_csv("data/mt_terminal_calls.csv")

In [ ]:
raw_data.head(2)

In [ ]:
raw_data.info()

##### lets analyse the ports a bit


In [ ]:
print(f'ships to HAMBURG : {raw_data.loc[raw_data["current_port"]=="HAMBURG",:].shape[0]}')
print(f'ships to LA : {raw_data.loc[raw_data["current_port"]!="HAMBURG",:].shape[0]}')

I will create a new feature: a ratio of GRT/(length*width)

In [ ]:
raw_data["doc_timestamp"] = pd.to_datetime(raw_data.loc[:, "doc_timestamp"])
raw_data["last_terminal_doc_timestamp"] = pd.to_datetime(raw_data.loc[:, "last_terminal_doc_timestamp"])


In [ ]:
raw_data["diff"] = raw_data.loc[:, "doc_timestamp"] - raw_data.loc[:, "last_terminal_doc_timestamp"]
raw_data["diff"] = raw_data.apply(lambda x: x['diff'].seconds,axis=1)

Some extra features

In [ ]:
raw_data["ratio"] = raw_data.loc[:, "grt"]/(raw_data.loc[:,"width"]*raw_data.loc[:,"length"])
raw_data["foot_ratio"] = raw_data.loc[:, "teu"]/(raw_data.loc[:,"width"]*raw_data.loc[:,"length"])

raw_data["size"] = raw_data.loc[:,"width"]*raw_data.loc[:,"length"]

ratio feature distribution

In [ ]:
raw_data.loc[raw_data["current_port"]=="HAMBURG","ratio"].hist(bins=20)

In [ ]:
raw_data.loc[raw_data["current_port"]!="HAMBURG","ratio"].hist(bins=20)

In [ ]:
raw_data.loc[:,"ratio"].hist(bins=20)

size feature distribution

In [ ]:
raw_data.loc[raw_data["current_port"]!="HAMBURG","size"].hist(bins=20)

In [ ]:
raw_data.loc[raw_data["current_port"]!="HAMBURG",["size"]].describe()

In [ ]:
raw_data.loc[:,"size"].hist(bins=20)

clearly much different distribution per port

In [ ]:
terminal_ham = raw_data.loc[raw_data["current_port"]=="HAMBURG", "current_terminal"].unique().tolist()
terminal_la = raw_data.loc[raw_data["current_port"]=="LOS ANGELES", "current_terminal"].unique().tolist()

In [ ]:
def id2port(x, th, tl):
    if x["current_port"] == "HAMBURG":
        idx = th[x["current_terminal"]]
    else:
        idx = tl[x["current_terminal"]]
    return idx

In [ ]:
terminal_ham = {i:idx for idx, i in enumerate(terminal_ham)}
terminal_la = {i:idx for idx, i in enumerate(terminal_la)}
raw_data["id_on_port"] =  raw_data.apply(id2port, args=(terminal_ham, terminal_la), axis=1)
la_distribution = raw_data.loc[raw_data["current_port"]!="HAMBURG", ["width","length","size","id_on_port"]]
# Create hexbin plot with Seaborn
sns.jointplot(data=la_distribution, x="width",y="length",hue="id_on_port",)

# Show the plot
plt.show()

In [ ]:
hamburg_distibution = raw_data.loc[raw_data["current_port"]=="HAMBURG", ["width","length","size","id_on_port"]]
# Create hexbin plot with Seaborn
sns.jointplot(data=hamburg_distibution, x="width",y="length",hue="id_on_port",  color="#4CB391")

# Show the plot
plt.show()

In [ ]:
nu_cols = raw_data.select_dtypes(include=['float64','int64']).columns
print(nu_cols)

In [ ]:
sns.pairplot(raw_data.loc[raw_data["current_port"]=="HAMBURG", nu_cols],hue="id_on_port", palette="Set2")

In [ ]:
sns.pairplot(raw_data.loc[raw_data["current_port"]!="HAMBURG", nu_cols],hue="id_on_port", palette="Set3")

# Construct baseline

#### use previous terminal as a prediction for the next

In [ ]:
gather_results = dict()
gather_data = dict()

In [ ]:
grouped_by_ship = raw_data.groupby(["shipname","current_port"])
restructure = []
matches = 0
for ship_no,(name,group) in enumerate(grouped_by_ship):
    restructure.append(group.sort_values(by="last_terminal_doc_timestamp"))
    restructure[-1].loc[:, "concat"] = restructure[-1].loc[:,"id_on_port"].astype(str) +"__" + restructure[-1].loc[:,"current_port"]
    restructure[-1].loc[:, "concat_s"] = restructure[-1].loc[:, "concat"].shift()
    tmp = restructure[-1].dropna()
    if tmp.dropna().shape[0]:
        matches += tmp.dropna().loc[restructure[-1]["concat"]==restructure[-1]["concat_s"], :].shape[0]
        

In [ ]:
matches, raw_data.shape[0]

In [ ]:
matches/raw_data.shape[0] # this here is essentially the accuracy metric on the whole dataset.

#### explanation of the main metrics that we will use since this problem is multi class classification problem.

Micro F1 and Macro F1 scores are two commonly used metrics to evaluate the performance of a multi-class classification model.

Micro F1 score considers all the predictions and ground truth values from all classes, and gives an equal weight to each sample. This means that the overall F1 score is calculated by taking into account the total true positives, false positives, and false negatives across all classes. Micro F1 is useful when you want to put equal emphasis on each individual sample in the evaluation.

Macro F1 score, on the other hand, calculates the F1 score for each class and takes the unweighted mean of the F1 scores. In this case, each class is given equal importance, regardless of the number of samples in each class. Macro F1 is useful when you care about the overall performance of the model across all classes, rather than the performance on individual samples.

So, to summarize, if you want to put equal emphasis on each sample, use Micro F1. If you care about the overall performance of the model across all classes, use Macro F1.

In [ ]:
restructure = pd.concat(restructure)
print(restructure.shape)

In [ ]:
population = Counter(restructure["concat"])
pprint(sorted([(value,key) for key, value in population.items()]))
print("HAMBURG")
pprint(sorted([(value,key) for key, value in population.items() if "HAM" in key]))
print("LA")
pprint(sorted([(value,key) for key, value in population.items() if "HAM" not in key]))

In our case we have a clear majority classifier especially on 4 of the HAMBURG ports, but our main objective is not to
place the ships equally in the port but rather predict their true destination. Hence we will give very slightly more weight to micro f1.

In [ ]:
print(restructure.shape[0])
restructure.loc[restructure["concat_s"].isna(), "concat_s"] = "no history"
print("macro f1",f1_score(restructure["concat"], restructure["concat_s"],average='macro'))
print("micro f1", f1_score(restructure["concat"], restructure["concat_s"],average='micro'))

This is the first and extremely naive baseline since it does not take into account the prior knowledge of the port

#### Next steps to reach the final solution

1. Verify/reject the markets team theory that historical data of the ship work.

2. Create more features by using polynomial ones in order to help catch a non linear relationship between the features and the actual label

3. In the procedure of solving the problem i will use essentially 19 possible labels ( 8 LA terminals + 11 HAMBURG terminals) and i will show all the increments before i reach my final result.

4. Reduce the set of labels by introducing the "port" feature as a binary feature (since we have a priori knowledge). This way we could simplify the problem, and in the case of a missclassification it could easily be rectified with postprocessing

### we need to create the feature that keeps the history track of the ship

In [ ]:
restructure.head(2)

In [ ]:
restructure["doc_timestamp"] = pd.to_datetime(restructure.loc[:, "doc_timestamp"])
restructure["last_terminal_doc_timestamp"] = pd.to_datetime(restructure.loc[:, "last_terminal_doc_timestamp"])

In [ ]:
grouped_by_ship = restructure.groupby(["shipname"])
# here *we don't* need to groupby the port as well, we just need the historical data.
make_history = []
matches = 0
for ship_no,(name,group) in enumerate(grouped_by_ship):
    group = group.sort_values(by="last_terminal_doc_timestamp")    
    increment = list()
    historical_data = group.loc[:, "concat"].to_list()
    trajectory = list()
    number_of_trips = list(range(len(historical_data)))
    for item in historical_data:
        if not len(increment):
            trajectory.append(["no history"])
        else:
            trajectory.append([])
            trajectory[-1].extend(increment)
        increment.append(item)
    group["history"] = trajectory
    group["no_trips"] = number_of_trips
    trips_on_the_current_port = [1 if y[-1] in x[-1] else 0 for x,y in zip(trajectory, list(group["current_port"]))]
    tptp = [1 if y[-1] in x[-1] else 0 for x,y in zip(trajectory, list(group["current_port"]))]
#     group["trips_on_this_port"] = [sum(tptp[:idx]) for idx, i in enumerate(tptp,1)]
    
    sm = list()
    for i in trajectory:
        if "no history" in i[-1] and len(i)==1:
            sm.append(0)
        elif "no history" not in i:
            sm.append(len(set(i)))
    group["different_terminals"] = sm#[0 if (('no history' in i[-1]) and (len(i[-1])==1)) else len(set(i[-1])) for i in trajectory ]
    group["last_visit"] = ["no history" if "history" in i[-1] else i[-1] for i in trajectory]
    group["last_port_visit"] = group["concat_s"].copy()
    group["parallel"] = historical_data
    make_history.append(group)
    

In [ ]:
make_history = pd.concat(make_history)

In [ ]:
encoder = LabelEncoder()
make_history["parallel"] = encoder.fit_transform(make_history["parallel"])

In [ ]:
make_history.head(2)

now we effectively/easily use this dataset or a subset of it (we don't need ALL the features anymore) and tackle this as a timeseries problem.

as we said before it is important to have the port as a feature since we're expected to know the port the ship is going

In [ ]:
make_history["current_port"] = make_history.apply(lambda x: 0 if "HAM" in x["current_port"] else 1,axis=1)

the last ports and the last terminals don't actually matter. What actually matters is the difference between the current port call and the previous port call.

In [ ]:
backup_history = make_history.copy()
# dataframe cleanup
del make_history["last_port"]
del make_history["last_terminal"]
# del make_history["doc_timestamp"]
del make_history["last_terminal_doc_timestamp"]
del make_history["concat_s"]

enc = OrdinalEncoder()
make_history.loc[:, ["shipname"]] = enc.fit_transform(make_history.loc[:, ["shipname"]])

In [ ]:
one_hot = pd.get_dummies(make_history.loc[:, "last_visit"])
one_hot_columns = list(one_hot.columns)

one_hot_2 = pd.get_dummies(make_history.loc[:, "last_port_visit"])
one_hot_2 = one_hot_2.add_suffix("oh2")
one_hot_columns_2 = list(one_hot_2.columns)


extra_columns = one_hot_columns +  one_hot_columns_2

make_history = pd.concat([make_history, one_hot, one_hot_2],axis=1)


this WILL NOT BE USED AS A FEATURE but rather as a medium to be able to process the data a bit easier

the shipname cannot be possibly used as a feature because
it will create curse of dimentonality since we can only use one hot
representations. Unless of course we could use ship embeddings (which would)
take a bigger dataset.

In [ ]:
make_history.loc[:, "last_visit"].unique()

In [ ]:
make_history.head(2)

In [ ]:
# x_tr, x_te, y_tr, y_te = [],[],[],[]
# for ship in make_history.loc[:,"shipname"].unique():
#     # they are already sorted by date
#     tmp = make_history.loc[make_history["shipname"]==ship,:].copy().sort_values(by="doc_timestamp")

#     X = tmp.loc[:,tmp.columns!="id_on_port"].copy()
#     del X["shipname"]
#     y = tmp.loc[tmp["shipname"]==ship, ["id_on_port"]].copy()

#     if X.shape[0]>=5: # anything less than 5 samples doesn't even matter if it get in the training dataset. (apart from the obvious fact that you cant split less than 5 samples)
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
#         x_tr.append(X_train)
#         x_te.append(X_test)

#         y_tr.append(y_train)
#         y_te.append(y_test)
#     else:
#         x_te.append(X)
#         y_te.append(y)


In [ ]:
make_history = make_history.sort_values(by="doc_timestamp")


In [ ]:
make_history["quarter"] = make_history.loc[:, "doc_timestamp"].dt.quarter

In [ ]:
make_history["day of week"] = make_history.loc[:, "doc_timestamp"].dt.dayofweek
make_history["year quarter"] = make_history.loc[:, "doc_timestamp"].dt.quarter

In [ ]:
X_train_main, X_test_main, y_train, y_test= train_test_split(make_history.loc[:,make_history.columns!="id_on_port"], make_history.loc[:,["id_on_port"]], test_size=0.2, shuffle=False)# pd.concat(x_tr), pd.concat(x_te), pd.concat(y_tr), pd.concat(y_te)

y_train_main = y_train.copy()
y_test_main = y_test.copy()

X_train_main and X_test_main is a superset of all the features so we can run with any subset. The whole dataset is splitted in a way to solve as a timeseries problem per ship (even if we don't use the ship as a feature)

In [ ]:
ft = ['no_trips','current_port', 'grt', 'teu', 'length', 'width', 'diff', 'ratio', 'size', 'different_terminals','foot_ratio']
print(f" features used: {ft}")

X_train = X_train_main.loc[:, ft].copy()
X_test = X_test_main.loc[:, ft].copy()

In [ ]:
X_train.loc[:, ['grt', 'teu', 'length', 'width', 'diff', 'ratio', 'foot_ratio', 'size',]].describe()

### phase 2 of experiments with these features

# logistic regression

In [ ]:
model = LogisticRegression(random_state=42)

svt = RobustScaler()

model.fit(svt.fit_transform(X_train),y_train)

svtest = RobustScaler()

y_pred = model.predict(svtest.fit_transform(X_test))

print("micro", f1_score(y_pred,y_test,average='micro'))
print("macro", f1_score(y_pred,y_test,average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))

gather_results["lr_ft"] = ({"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')})

gather_data["lr_ft"] = {"X_train":svt.fit_transform(X_train),"y_train":y_train,"X_test":svtest.fit_transform(X_test),"y_test":y_test,"model":model, "y_pred":y_pred}


# Random Forest Classifier

In [ ]:
model = RandomForestClassifier(random_state=42)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["rfr_ft"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["rfr_ft"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred)

In [ ]:
check_df = pd.DataFrame(model.feature_importances_.reshape(1,-1), columns=X_train.columns).T
check_df.columns = ["importance"]

In [ ]:
check_df.sort_values(by="importance")

# SVC

In [ ]:
# SVMs' need scaling before used.
model = SVC(random_state=42)


svt = RobustScaler()

model.fit(svt.fit_transform(X_train),y_train)

svtest = RobustScaler()

y_pred = model.predict(svtest.fit_transform(X_test))

print("micro", f1_score(y_pred.ravel(),y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred.ravel(),y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["svm_ft"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}


gather_data["svm_ft"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred)

In [ ]:
checking_df = pd.concat([X_test,y_test],axis=1)

In [ ]:
checking_df.loc[(checking_df["current_port"]==1) & (checking_df["id_on_port"]>7),:]

Of course here we have cases of misclassification, and using post process we can place correct the prediction of the 9 and above.
Terminal for port of Los Angeles to any of the other 8 ports that are available.

1. This could happen in multiple ways, either by using the knowledge of the prior distribution and place the prediction of over 8th terminal to the most visited terminal of the correct port.
2. If we had information about the terminals (aka similiarties of the terminals i would try to correlate the 9 to 11 terminals to one of the other 8.



# compare with baseline on the same validation set.

In [ ]:
restructure2 = restructure.loc[restructure.index.isin(X_test.index), :].copy()
print(restructure2.shape[0])
print("macro f1",f1_score(restructure2["concat"], restructure2["concat_s"],average='macro'))
print("micro f1", f1_score(restructure2["concat"], restructure2["concat_s"],average='micro'))

gather_results["base_vali"] = {"micro f1":f1_score(restructure2["concat"], restructure2["concat_s"],average='micro'), "macro":f1_score(restructure2["concat"], restructure2["concat_s"],average='macro')}

print(classification_report(restructure2["concat_s"], restructure2["concat"]))

In [ ]:
restructure2.loc[restructure2["concat_s"]=="no history",:].head(2)

In [ ]:
# one of the reasons that we get a better result from the baseline is that we do have a majority classifier on our hands 

# Evidently we need to explore new features

In [ ]:
# first of all we must definitely should use the last_known terminal
# hence veifying the hypothesis of the product team.

In [ ]:
X_train_main.shape

In [ ]:
X_train_main.loc[X_train_main["last_visit"]=="no history",:].head(2)

In [ ]:
X_train_main.loc[X_train_main["last_visit"]=="no history",:].shape[0],X_test_main.loc[X_test_main["last_visit"]=="no history",:].shape[0]

should i use the previous lag i would have to delete 685 + 712 rows out of the dataset since they don't actually have history, instead i have decided to make that a feature.

In [ ]:
expanded_cols = ft.copy()
expanded_cols.extend(one_hot_columns)
expanded_cols.extend(one_hot_columns_2)
X_train = X_train_main.loc[:, expanded_cols].copy()
X_test = X_test_main.loc[:, expanded_cols].copy()

### Logistic regression with expanded set of features.

In [ ]:
model = LogisticRegression(random_state=42)

svt = RobustScaler()

model.fit(svt.fit_transform(X_train),y_train)

svtest = RobustScaler()

y_pred = model.predict(svtest.fit_transform(X_test))

print("micro", f1_score(y_pred,y_test,average='micro'))
print("macro", f1_score(y_pred,y_test,average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))

gather_results["lr_expanded"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["lr_expanded"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred)

### RandomForestClassifier with expanded set of features.

In [ ]:
model = RandomForestClassifier(random_state=42)

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["rfr_expanded"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["rfr_expanded"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred)

In [ ]:
md = RandomForestClassifier(random_state=42)
from sklearn.model_selection import TimeSeriesSplit
tscv = TimeSeriesSplit()
tscv = TimeSeriesSplit(n_splits=3, test_size=round(0.1*len(X_train)), gap=2)

plot_learning_curve(md, X=X_train, y=y_train, scoring='f1_micro', cv=tscv, shuffle=False)


In [ ]:
# it seems that with more training examples the model actually learns.

### SVMClassifier with expanded set of features.

In [ ]:
# SVMs' need scaling before used.
model = SVC(random_state=42)


svt = RobustScaler()

model.fit(svt.fit_transform(X_train),y_train)

svtest = RobustScaler()

y_pred = model.predict(svtest.fit_transform(X_test))

print("micro", f1_score(y_pred.ravel(),y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred.ravel(),y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["svm_expanded"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["svm_expanded"] = {"X_train":svt.fit_transform(X_train),"y_train":y_train,"X_test":svtest.fit_transform(X_test),"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred)

All of those models could use hyperparameter tuning by using GridSearchCV (with cv disabled because we have a sort of a timeseries). But i thought it would make more sense to produce more features in order to get more insights for the problem. The EDA process is more valuable while tuning can be fully automated.

We can expand our feature search a bit more by using Polynomial features (without the bias) and clustering of the features
to give us an extra insight for data of our ships




In [ ]:
# Create a list of inertia values for different k values
inertias = []
k_values = range(1, 11)
for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_train)
    inertias.append(kmeans.inertia_)

# Plot the elbow graph
plt.plot(k_values, inertias, '-o')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method for KMeans Clustering')
plt.xticks(np.arange(1, 11))
plt.show()

as we can see from the plot the best number of clusters seems to be number 3 (it looks more like an elbow).

so we will test this extra feature on our best performing algorithm so far.

In [ ]:
print(X_train_main.shape,y_train_main.shape)

kmeans = KMeans(n_clusters=3, random_state=42)
res = kmeans.fit_predict(X_train)
X_train["res"] = res

tmp1 = pd.get_dummies(X_train["res"])
tmp1 = tmp1.add_suffix("oh2")

X_train = pd.concat([X_train,tmp1],axis=1)
del X_train["res"]

res = kmeans.predict(X_test)
X_test["res"] = res
tmp2 = pd.get_dummies(X_test["res"])
tmp2 = tmp2.add_suffix("oh2")
X_test = pd.concat([X_test,tmp2],axis=1)
del X_test["res"]

model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["rfr_cluster"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}


gather_data["rfr_cluster"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}


Same test for polynomial features. The polynomial features help in capturing non linear relationships.


### add Polynomial features + one hot encoding features + clusters

In [ ]:
print("features used")
print(list(X_test.columns))
degree = 2

# Create a PolynomialFeatures object
poly = PolynomialFeatures(degree, include_bias=False)

# Fit and transform the training data
X_train_poly = poly.fit_transform(X_train.loc[:, ft])

X_train_poly = pd.concat([pd.DataFrame(X_train_poly).reset_index(drop=True),X_train.loc[:, extra_columns].reset_index(drop=True),tmp1.reset_index(drop=True)],axis=1)
X_train_poly.columns = [f"lala_{i}" for i in list(X_train_poly.columns)]
# Transform the test data using the same polynomial features
X_test_poly = poly.transform(X_test.loc[:, ft])
X_test_poly = pd.concat([pd.DataFrame(X_test_poly),X_test.loc[:, extra_columns].reset_index(drop=True),tmp2.reset_index(drop=True)],axis=1)
X_test_poly.columns = X_train_poly.columns
model = RandomForestClassifier(random_state=42)
print(X_train_poly.shape)
model.fit(X_train_poly,y_train.astype(float))

y_pred = model.predict(X_test_poly)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["rfr_poly_cluster_expanded"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}



gather_data["rfr_poly_cluster_expanded"] = {"X_train":X_train_poly,"y_train":y_train,"X_test":X_test_poly,"y_test":y_test,"model":model, "y_pred":y_pred}


### add Polynomial features + one hot encoding features


In [ ]:
degree = 2

# Create a PolynomialFeatures object
poly = PolynomialFeatures(degree, include_bias=False)

# Fit and transform the training data
X_train_poly = poly.fit_transform(X_train.loc[:, ft])

X_train_poly = pd.concat([pd.DataFrame(X_train_poly).reset_index(drop=True),X_train.loc[:, extra_columns].reset_index(drop=True)],axis=1)
X_train_poly.columns = [f"lala_{i}" for i in list(X_train_poly.columns)]
# Transform the test data using the same polynomial features
X_test_poly = poly.transform(X_test.loc[:, ft])
X_test_poly = pd.concat([pd.DataFrame(X_test_poly),X_test.loc[:, extra_columns].reset_index(drop=True)],axis=1)
X_test_poly.columns = X_train_poly.columns
model = RandomForestClassifier(random_state=42)
print(X_train_poly.shape)
model.fit(X_train_poly,y_train.astype(float))

y_pred = model.predict(X_test_poly)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test.values.ravel()))
gather_results["rfr_expanded_poly"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["rfr_expanded_poly"] = {"X_train":X_train_poly,"y_train":y_train,"X_test":X_test_poly,"y_test":y_test,"model":model, "y_pred":y_pred}


# Finally we can perform feature selection

Using backward and forward elimination we can get the best subset of features that we can use. The thing is this procedure is extremely computationally expensive and i didn't manage the get the result of those functions.

In [ ]:
def backward_elimination(X, y, model, metric):
    best_features = list(X.columns)
    best_score = 0
    
    for k in tqdm(range(1, len(X.columns) + 1)):
        for subset in combinations(X.columns, k):
            model.fit(X.loc[:, list(subset)], y)
            y_pred = model.predict(X[list(subset)])
            score = metric(y, y_pred.ravel(), average='macro')
            
            if score > best_score:
                best_score = score
                best_features = list(subset)
                
    return best_features

def forward_selection(X, y, model, metric):
    best_features = []
    best_score = 0
    
    for k in tqdm(range(1, len(X.columns) + 1)):
        if k == 1:
            candidate_features = list(X.columns)
        else:
            candidate_features = [f for f in X.columns if f not in best_features]
        
        for feature in candidate_features:
            model.fit(X[best_features + [feature]], y)
            y_pred = model.predict(X[best_features + [feature]])
            score = metric(y, y_pred.ravel(), average='macro')
            
            if score > best_score:
                best_score = score
                best_features.append(feature)
                
    return best_features

### the product theory has been verified that the historical data been extremely useful on the next prediction.

In [ ]:
# put example of non timeseries data here.

In [ ]:
raw_data.columns

In [ ]:
raw_data.head(2)

In [ ]:
raw_data["concat"] = raw_data["current_port"] + raw_data["current_terminal"]

X = restructure.loc[:, ["grt","teu","length", "width", "diff", "ratio"]].copy()
Y = restructure.loc[:, ["concat"]].copy()

encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("micro f1",f1_score(y_test, y_pred, average="micro"))
print("macro f1",f1_score(y_test, y_pred, average="macro"))

In [ ]:
# keep the indices and compare with previous baseline

In [ ]:
restructure = restructure.loc[restructure.index.isin(X_test.index), :]
print("micro f1", f1_score(restructure["concat_s"],restructure["concat"],average='micro' ))
print("macro f1",f1_score(restructure["concat_s"], restructure["concat"],average='macro' ))


In [ ]:
X_test.shape, restructure.shape

In [ ]:
print(classification_report(restructure["concat_s"], restructure["concat"]))

# labels distribution

0. (1, '10HAMBURG')
1. (1, '7LOS ANGELES')
2. (1, '9HAMBURG')
3. (9, '8HAMBURG')
4. (28, '7HAMBURG')
5. (40, '6HAMBURG')
6. (157, '4HAMBURG')
7. (225, '2LOS ANGELES')
8. (266, '1HAMBURG')
9. (271, '4LOS ANGELES')
10. (323, '6LOS ANGELES')
11. (355, '5LOS ANGELES')
12. (469, '3LOS ANGELES')
13. (485, '1LOS ANGELES')
14. (514, '0LOS ANGELES')
15. (1183, '5HAMBURG')
16. (2313, '2HAMBURG')
17. (2574, '0HAMBURG')
18. (2839, '3HAMBURG')

##### check with  19 labels on best performing models

In [ ]:
model = gather_data["svm_expanded"]["model"]

X_train = gather_data["svm_expanded"]["X_train"]
X_test = gather_data["svm_expanded"]["X_test"]

y_train = X_train_main.loc[:,"parallel"]
y_test = X_test_main.loc[:,"parallel"]

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))

gather_results["svm_19_labels"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

gather_data["svm_19_labels"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}


print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))

heatmap(y_test, y_pred, 19)

In [ ]:
print(len(ft))
sc = RobustScaler()
X = X_train_main.loc[:, ft].copy().copy()
X = sc.fit_transform(X)
y = X_train_main.loc[:, "parallel"].copy().copy()

predictor = model
backward_best_features = backward_elimination(X, y, predictor, f1_score)
print("Best features using backward elimination:", backward_best_features)

forward_best_features = forward_selection(X, y, predictor, f1_score)
print("Best features using forward selection:", forward_best_features)

In [ ]:
model = gather_data["lr_expanded"]["model"]

X_train = gather_data["lr_expanded"]["X_train"]
X_test = gather_data["lr_expanded"]["X_test"]

y_train = X_train_main.loc[:,"parallel"]
y_test = X_test_main.loc[:,"parallel"]

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))

gather_results["lr_expanded_19_labels"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))


gather_data["lr_expanded_19_labels"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred, 19)

In [ ]:
model = gather_data["rfr_poly_cluster_expanded"]["model"]

X_train = gather_data["rfr_poly_cluster_expanded"]["X_train"]
X_test = gather_data["rfr_poly_cluster_expanded"]["X_test"]

y_train = X_train_main.loc[:,"parallel"]
y_test = X_test_main.loc[:,"parallel"]

model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))

gather_results["rfr_poly_cluster_expanded_19_labels"] = {"micro f1":f1_score(y_pred,y_test,average='micro'), "macro":f1_score(y_pred,y_test,average='macro')}

print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))

gather_data["rfr_poly_cluster_expanded_19_labels"] = {"X_train":X_train,"y_train":y_train,"X_test":X_test,"y_test":y_test,"model":model, "y_pred":y_pred}

heatmap(y_test, y_pred, 19)

##### best model so far

In [ ]:
best_model = pd.DataFrame.from_dict(gather_results).T.sort_values(by="macro",ascending=False).index[0]

In [ ]:
print(f"{best_model} is the best model so far")

In [ ]:
model = gather_data[best_model]["model"]

X_train = gather_data[best_model]["X_train"]#.loc[:, backward_best_features+expanded_cols]
# X_train = pd.concat([X_train, tmp1],axis=1)
X_test = gather_data[best_model]["X_test"]#.loc[:, backward_best_features+expanded_cols]
# X_test = pd.concat([X_test, tmp2],axis=1)
y_train = gather_data[best_model]["y_train"]
y_test = gather_data[best_model]["y_test"]
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))

In [ ]:
model = gather_data[best_model]["model"]

X_train = gather_data[best_model]["X_train"].loc[:, forward_best_features+expanded_cols]
X_train = pd.concat([X_train, tmp1],axis=1)
X_test = gather_databest_model]["X_test"].loc[:, forward_best_features+expanded_cols]
X_test = pd.concat([X_test, tmp2],axis=1)
y_train = gather_data[best_model]["y_train"]
y_test = gather_data[best_model]["y_test"]
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))

In [ ]:
model = gather_data["rfr_expanded"]["model"]

X_train = gather_data["rfr_expanded"]["X_train"]
X_test = gather_data["rfr_expanded"]["X_test"]
y_train = gather_data["rfr_expanded"]["y_train"]
y_test = gather_data["rfr_expanded"]["y_test"]
model.fit(X_train,y_train)

y_pred = model.predict(X_test)

print("micro", f1_score(y_pred, y_test.values.ravel(),average='micro'))
print("macro", f1_score(y_pred, y_test.values.ravel(),average='macro'))
print(classification_report(y_pred.ravel(),y_test_main.values.ravel()))

In [ ]:
to_compare = y_test.copy()
to_compare["predictions"] = y_pred.tolist()
wrong_preds = to_compare.loc[to_compare["id_on_port"]!=to_compare["predictions"],:].index
to_compare["current_port"] = X_test.loc[X_test.index.isin(wrong_preds),["current_port",]]
to_compare = to_compare.dropna()
print(f"{to_compare.shape[0]} were missclassified but it wasn't due to the final number of categories max(port_HAM,port_LA)")

Hamburg is 0 LA is 1

In [ ]:
to_compare.loc[to_compare["current_port"]==0,:]

In [ ]:
to_compare.loc[to_compare["current_port"]==0,"predictions"].unique()

In [ ]:
to_compare.loc[to_compare["current_port"]==1,:]

In [ ]:
to_compare.loc[(to_compare["current_port"]==1)& (to_compare["predictions"]>7),:]

In [ ]:
to_compare.loc[to_compare["current_port"]==1,"predictions"].unique()

In [ ]:
# something that completely makes sense since the examples for the terminal 7 and above for each port
# have extremely low occurence.

# Essentially that plays a role on the misbehaviors of the best performing model so far.

# Conclusions

1 & 2. The features finally selected are coming from the backwards and the forward eliminations. 
Correlation of the features did give an insight along with logistic regression coefficients and random forests' feature importances, but essentially *correlation does not imply causation* so the safest way to decide was FE and BE.

3. My findings agree with the product team since there was a better result by using the actual historical data of each ship to predict the next destination. Before build the model you must check the relationship between the variables as well as their relationship with their target (independent variable).

4. It is evident that this is a pure classification problem since we're called to predict essentially a categorical variable.

5. Overall my model has a better accuracy than the baseline (thus proving there was room for improvement) but it had worse results from the baseline on getting better results *across all of the terminals*

6. The reasons that the algorithm has failed in several occasions is that there were categories with extremely low pupulation i.e. some terminals had up to 10 occurences of data whereas the biggest occurences were found on terminals with over 2000 visits.
Also i had 1 model for both LA and HAMBURG so there were bound to be some missclassifications i.e. over the 9nth terminal in LA. Especially since there were significantly less examples in the LA port. Of course post processing here
helps the results. One other way that i could have tackled this issue is to actually have 2 models but i believe the one model actually found correlations between some of the terminals for each port.

7. The most naive approach i could find here is to actually use the previous visit on the current port as the prediction. Something which proved to be an excellent baseline.

## Possible future work

1. We could try ship embeddings.
2. Find a way to see if there is a cap on the occupation of each terminal so we can significantly reduce the search space of our predictions in production.
3. Even though i used the "diff" feature in the training, it would also be important to get the actual km distance from beggining to the end port.
4. We could try binning the "volume" of the ships that comes from a histogram as an extra feature.
5. Use extensive tuning on non linear ml algorithms but not DNNs since the dataset is quite small.
6. Post process by either performing clustering on the terminals (i.e. in case of invalid classification correct it with the closest valid terminal in the same cluster)

A bonus point for discussion:

Given the accuracy of your model, what would be the next steps in terms of deploying a solution such
that the model’s value to the business is maximised?


The first steps i would consider to maximize the business eficiency are:

1. Identify how to integrate the model into our workflow and actually set a shadow deployment for it.
2. Make sure that we use the proper tools for scaling it for larger datasets (and requests).
3. Evaluate the model in the inference by setting up an evaluation platform (this cannot be done manually for a long time).
4. Get feedback from the product on any tweaks that it might need to satisfy our customers.
5. Set up a plan to make sure it is futureproof and maintainable.
